# A/B tests с Python




<a id="Libraries"></a>
##  Загрузка библиотек 

In [1]:
from math import asin
from typing import Union


import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

# Поработаем с датасетом c покупками

# Подготовка данных

In [2]:
data = pd.read_csv('ab_stats.csv')

data.head(10)

,revenue,num_purchases,purchase,ab_group,av_site visit
0,0.0,0,0,A,9.040174
1,0.0,0,0,A,4.811628
2,0.0,0,0,A,7.342623
3,0.0,0,0,A,7.744581
4,0.0,0,0,A,10.511814
5,0.0,0,0,A,9.578727
6,0.0,0,0,A,6.162601
7,0.0,0,0,A,11.909452
8,0.0,0,0,A,6.540910
9,0.0,0,0,A,7.990794


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23652 entries, 0 to 23651
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   revenue        23652 non-null  float64
 1   num_purchases  23652 non-null  int64  
 2   purchase       23652 non-null  int64  
 3   ab_group       23652 non-null  object 
 4   av_site visit  23652 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 924.0+ KB


In [4]:
data.shape

(23652, 5)

In [5]:
data.describe()

,revenue,num_purchases,purchase,av_site visit
count,23652.000000,23652.000000,23652.000000,23652.000000
mean,0.324689,0.043590,0.020717,7.013112
std,9.557730,1.079403,0.142438,3.154584
min,0.000000,0.000000,0.000000,-12.073486
25%,0.000000,0.000000,0.000000,5.173787
50%,0.000000,0.000000,0.000000,7.007936
75%,0.000000,0.000000,0.000000,8.864119
max,1303.609284,152.000000,1.000000,22.446822


Посмотрим на группы отдельно

In [6]:
control = data[data['ab_group'] == 'A'].copy(deep=True)
treatment = data[data['ab_group'] == 'B'].copy(deep=True)

In [7]:
control.describe()

,revenue,num_purchases,purchase,av_site visit
count,11835.000000,11835.000000,11835.000000,11835.000000
mean,0.404462,0.050697,0.021631,6.974724
std,13.133218,1.467511,0.145481,2.023533
min,0.000000,0.000000,0.000000,-12.073486
25%,0.000000,0.000000,0.000000,5.656155
50%,0.000000,0.000000,0.000000,6.982329
75%,0.000000,0.000000,0.000000,8.345572
max,1303.609284,152.000000,1.000000,17.728836


In [8]:
treatment.describe()

,revenue,num_purchases,purchase,av_site visit
count,11817.000000,11817.000000,11817.000000,11817.000000
mean,0.244794,0.036473,0.019802,7.051559
std,3.176534,0.418480,0.139325,3.976799
min,0.000000,0.000000,0.000000,-8.286822
25%,0.000000,0.000000,0.000000,4.380984
50%,0.000000,0.000000,0.000000,7.060873
75%,0.000000,0.000000,0.000000,9.768648
max,113.830000,25.000000,1.000000,22.446822


Пока мы еще ничего не посчитали, но уже можно заметить, что максимальный чек в первой группе сильно больше, чем в группе B. Все мы знаем, что среднее очень неустойчиво к выбросам, так что нам необходимо будет это учесть.

# Проверка на нормальность распределения и применение статистических критериев

In [9]:
def is_normal_distr(x):
    
    if x.size < 5_000:
        _, pvalue = stats.shapiro(x)
    else:
        _, pvalue = stats.kstest(x, 'norm')
        
    return pvalue

In [10]:
def calc_continuous_effect(control: pd.DataFrame,
                           treatment: pd.DataFrame,
                           column: str,
                           stat_test: Union['t', 'mw'] = 't') -> pd.DataFrame:
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    nobs1 = control.shape[0]
    nobs2 = treatment.shape[0]
    
    # effect_size = (treatment_mean - control_mean) / ((control_std ** 2 + treatment_std ** 2) / 2) ** .5
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=nobs2,
                                    mean2=control_mean, sd2=control_std, nobs2=nobs1)
    
    if stat_test == 't':
        _, pvalue = stats.ttest_ind(a=control.loc[:, column],
                                    b=treatment.loc[:, column],
                                    equal_var=False, # perform Welch's t-test
                                    alternative='two-sided')
   
    elif stat_test == 'mw':
        _, pvalue = stats.mannwhitneyu(x=control.loc[:, column],
                                       y=treatment.loc[:, column],
                                       alternative='two-sided')
    else:
        raise NotImplementedError()

    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=control.shape[0],
                               alpha=pvalue,
                               power=None,
                               ratio=nobs2/nobs1)
    
    pw_settings = {'alpha': .05, 'power': .8}
    pw_nobs = tt_ind_solve_power(effect_size=effect_size,
                                 nobs1=None,
                                 alpha=pw_settings['alpha'],
                                 power=pw_settings['power'],
                                 ratio=1)
    
    difference = treatment_mean - control_mean
    
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': pvalue, 
                           'beta': (1-power),
                           'power': power,
                           'difference': difference,
                           'nobs': nobs1 + nobs2},
                          index=[column]) 
    
    perfect_way = pd.DataFrame({'effect_size': effect_size,
                                'alpha': pw_settings['alpha'],
                                'beta': 1 - pw_settings['power'],
                                'power': pw_settings['power'],
                                'difference': difference,
                                'nobs': round(pw_nobs * 2, 0)},
                               index=['perfect_way'])
    
    return pd.concat((result, perfect_way))

In [11]:
def calc_proportion_effect(control: pd.DataFrame,
                           treatment: pd.DataFrame,
                           column: str,) -> pd.DataFrame:
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    conv1 = control.loc[:, column].sum()
    conv2 = treatment.loc[:, column].sum()
    
    nobs1 = control.shape[0]
    nobs2 = treatment.shape[0]
    
    # effect_size_f = 2  * asin(np.sqrt(conv1/nobs1)) - 2 * asin(np.sqrt(conv2/nobs2))
    effect_size = proportion.proportion_effectsize(prop1=conv1/nobs1, prop2=conv2/nobs2)
    
    _, chi_pvalue, _ = proportion.proportions_chisquare([conv1, conv2], [nobs1, nobs2])
    
    _, z_pvalue = proportion.proportions_ztest([conv1, conv2], [nobs1, nobs2])

    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=nobs1,
                               alpha=z_pvalue,
                               power=None,
                               ratio=nobs2/nobs1)
    
    pw_settings = {'alpha': .05, 'power': .8}
    pw_nobs = zt_ind_solve_power(effect_size=effect_size,
                                 nobs1=None,
                                 alpha=pw_settings['alpha'],
                                 power=pw_settings['power'],
                                 ratio=1)
    
    difference = treatment_mean - control_mean
    
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha_chi': chi_pvalue, 
                           'alpha_z': z_pvalue,
                           'beta': (1-power),
                           'power': power,
                           'difference': difference,
                           'nobs': nobs1 + nobs2},
                          index=[column]) 
    
    perfect_way = pd.DataFrame({'effect_size': effect_size,
                                'alpha_chi': pw_settings['alpha'],
                                'alpha_z': pw_settings['alpha'],
                                'beta': 1 - pw_settings['power'],
                                'power': pw_settings['power'],
                                'difference': difference,
                                'nobs': round(pw_nobs * 2, 0)},
                               index=['perfect_way'])
    
    return pd.concat((result, perfect_way))

## Метрика выручка на юзера с покупкой

In [24]:
fig = px.histogram(data[data['purchase'] == 1],
                   x='revenue',
                   color = 'ab_group',
                   title='revenue_distribution',
                   marginal = 'box',
                   nbins = 50)
fig.show()

In [25]:
column = 'revenue'
control_is_normal = is_normal_distr(control[control.purchase == 1].loc[:, column])
treatment_is_normal = is_normal_distr(treatment[treatment.purchase == 1].loc[:, column])
control_is_normal, treatment_is_normal

(2.4723388038366e-32, 1.599590675565433e-22)

Данное распределение не считаем нормальным

In [23]:
calc_continuous_effect(control[control.purchase == 1], 
                       treatment[treatment.purchase == 1],
                       column='revenue', stat_test='mw')

,effect_size,alpha,beta,power,difference,nobs
revenue,-0.097905,0.887196,0.063033,0.936967,-6.33635,490.0
perfect_way,-0.097905,0.050000,0.200000,0.800000,-6.33635,3277.0


Альфа 0,88 - нет статзначимых различий. Недостаточное количество наблюдений<p>
Нужно изменить формулировку гипотезы или выюрать другую метрику и провести новый тест